In [2]:
import numpy as np 
import pandas as pd 

countries_report = pd.read_csv("countries.csv", header = 0, low_memory=False)
happiness_report = pd.read_csv("2017.csv", header = 0, low_memory=False)

happiness_report = happiness_report.sample(frac=1)
print happiness_report.shape
print countries_report.shape

(151, 12)
(227, 20)


In [3]:
happy_data = []
country_data = []

rows = happiness_report.shape[0]
for i in range(rows):
    key = unicode("Country", "utf-8")
    c = happiness_report.loc[happiness_report.index[[i]], key]
    c_data = happiness_report.loc[happiness_report.index[[i]]]
#     print c_data.values[0]
    happy_data.append(c_data.values[0])
    
    # Gets corresponding countries.csv data for each country in 2017.ccsv
    Country = happiness_report.loc[happiness_report.index[[i]], key].values
#     print Country[0]
    
    col = countries_report['Country']
    index = -1
    for i in col:
        index += 1
        i = str(i)
        if Country[0] in i:
#             print i
            idx = index
            break
        
    data = countries_report.loc[countries_report.index[idx]]
    country_data.append(data)

In [4]:
print len(happy_data)
print len(country_data)

#sanity check
idx = 0
for i in country_data:
    print happy_data[idx][0],
    print i[0]
    idx += 1

151
151
Romania Romania 
Germany Germany 
Georgia Georgia 
Armenia Armenia 
Norway Norway 
Liberia Liberia 
Japan Japan 
Togo Togo 
Macedonia Macedonia 
Serbia Serbia 
Venezuela Venezuela 
Bhutan Bhutan 
Egypt Egypt 
Saudi Arabia Saudi Arabia 
Kazakhstan Kazakhstan 
Malta Malta 
Lesotho Lesotho 
India India 
France France 
Cote d'Ivoire  Cote d'Ivoire 
Iceland Iceland 
Spain Spain 
Thailand Thailand 
Indonesia Indonesia 
Slovakia Slovakia 
Albania Albania 
Mauritania Mauritania 
Finland Finland 
Niger Niger 
Cambodia Cambodia 
Libya Libya 
Israel Israel 
Portugal Portugal 
Angola Angola 
Uzbekistan Uzbekistan 
Honduras Honduras 
El Salvador El Salvador 
Nigeria Nigeria 
Haiti Haiti 
Nepal Nepal 
Ukraine Ukraine 
Namibia Namibia 
Belarus Belarus 
Poland Poland 
Moldova Moldova 
Yemen Yemen 
Turkmenistan Turkmenistan 
Brazil Brazil 
Sudan Sudan 
Benin Benin 
Dominican Republic Dominican Republic 
Madagascar Madagascar 
Russia Russia 
Bosnia and Herzegovina Bosnia and Herzegovina 
Malawi 

In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# print happy_data
happyX = []
happyY = []

for i in happy_data:
    happyX.append(i[0])
    happyY.append(i[2])
    
happyX = np.array(happyX)
happyY = np.array(happyY)
print happyX.shape
print happyY.shape

def mse_baseline(y_train, y_test):
    avg = np.average(y_train)
    print avg
    y_predict = np.full(y_test.shape, avg)
    print "baseline", mean_squared_error(y_test, y_predict)

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(happyX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = happyX[train_index], happyX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    mse_baseline(y_train, y_test)

(151,)
(151,)
5.4159600043
baseline 1.28527711066
5.33679207716
baseline 1.33137868782
5.34958415696
baseline 1.2433831374


In [6]:
import math
allArr = np.zeros(19)
for i in country_data:
    for j in range(2,19):
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                allArr[j] += i[j]
        else:
            allArr[j] +=(float(i[j].replace(',','.')))
means = allArr/float(len(country_data))

In [22]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

countriesX = [] 

# train using one column from 
# for i in country_data:
#     #comment this line out if not float data
#     f = float(i[7].replace(',','.'))
#     countriesX.append([f])

regions = []
for i in country_data:
    regions.append(i[1])

le = preprocessing.LabelEncoder()
regions_codes = le.fit_transform(regions)

for num, i in enumerate(country_data):
    info = []
#     info.append(regions_codes[num])
    for j in [7, 8, 10]: 
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                info.append(i[j])
            else:
                info.append(means[j])
        else:
            info.append(float(i[j].replace(',','.')))  
    countriesX.append(info)
    
countriesX = np.array(countriesX)
print countriesX.shape
print happyY.shape
# lab_enc = preprocessing.LabelEncoder()
# happyY = lab_enc.fit_transform(happyY)

    
clusters=6
km = KMeans(n_clusters=clusters)
km.fit(countriesX)
test_clusters = km.predict(countriesX)

(151, 3)
(151,)


In [39]:
sums = np.zeros(clusters)
counts = np.zeros(clusters)
averages = np.zeros(clusters)
    
for i, label in enumerate(km.labels_):
    sums[label] += happyY[i]
    counts[label] += 1
    
for i in range(clusters):
    averages[i] = sums[i]/counts[i]
    
least_sim = -1
similarity = -1 
for i, label in enumerate(km.labels_):
    if label == 5:
        if abs(happyY[i] - averages[5]) > similarity:
            least_sim = i
            similarity = abs(happyY[i] - averages[5])
print country_data[least_sim][0]
print averages[5]
print happyY[least_sim]

Central African Republic
4.41430000065
2.693000078
